# CNN Case Study: Butterfly Species Classification

## Project Overview

In this case study, you will build a Convolutional Neural Network (CNN) to classify images of butterflies and moths into **100 different species**. This is a comprehensive deep learning project that covers:

- Data loading and preprocessing
- Exploratory data analysis
- Building CNN architectures from scratch
- Transfer learning with pre-trained models
- Model evaluation and visualization
- Hyperparameter tuning

### Dataset Information

- **Source**: [Kaggle - Butterfly Images (100 species)](https://www.kaggle.com/datasets/gpiosenka/butterfly-images40-species)
- **Total Images**: 13,595
- **Image Size**: 224 × 224 × 3 (RGB)
- **Classes**: 100 butterfly and moth species
- **Split**: 
  - Training: 12,594 images
  - Validation: 500 images (5 per species)
  - Test: 500 images (5 per species)

### Learning Objectives

By completing this case study, you will:
1. Understand how to work with image datasets
2. Build and train CNNs using TensorFlow/Keras
3. Implement data augmentation techniques
4. Compare different model architectures
5. Apply transfer learning
6. Evaluate model performance using various metrics

---
## 1. Setup and Imports

In [ ]:
# Data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Scikit-learn for metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Display settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

---
## 2. Data Loading and Exploration

### 2.1 Download the Dataset

First, download the dataset from Kaggle. You'll need to:
1. Install Kaggle API: `pip install kaggle`
2. Set up your Kaggle API credentials
3. Download the dataset

In [ ]:
# Uncomment and run these lines to download the dataset
# !pip install kaggle
# !kaggle datasets download -d gpiosenka/butterfly-images40-species
# !unzip butterfly-images40-species.zip -d butterfly_data

In [ ]:
# Define data paths - UPDATE THESE PATHS according to your setup
BASE_DIR = Path('butterfly_data')  # Update this to your data directory
TRAIN_DIR = BASE_DIR / 'train'
VALID_DIR = BASE_DIR / 'valid'
TEST_DIR = BASE_DIR / 'test'

# Image parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
BATCH_SIZE = 32

print(f"Train directory exists: {TRAIN_DIR.exists()}")
print(f"Valid directory exists: {VALID_DIR.exists()}")
print(f"Test directory exists: {TEST_DIR.exists()}")

### 2.2 Explore the Dataset Structure

In [ ]:
# Count images in each set
def count_images(directory):
    """Count total images and images per class in a directory."""
    total = 0
    class_counts = {}
    
    for class_dir in sorted(directory.iterdir()):
        if class_dir.is_dir():
            num_images = len(list(class_dir.glob('*.jpg')))
            class_counts[class_dir.name] = num_images
            total += num_images
    
    return total, class_counts

# Count images in each set
train_total, train_counts = count_images(TRAIN_DIR)
valid_total, valid_counts = count_images(VALID_DIR)
test_total, test_counts = count_images(TEST_DIR)

print(f"Training images: {train_total}")
print(f"Validation images: {valid_total}")
print(f"Test images: {test_total}")
print(f"\nNumber of classes: {len(train_counts)}")
print(f"\nFirst 10 species: {list(train_counts.keys())[:10]}")

### 2.3 Visualize Sample Images

In [ ]:
def plot_sample_images(directory, num_samples=12, figsize=(15, 10)):
    """Plot random sample images from the dataset."""
    classes = sorted([d.name for d in directory.iterdir() if d.is_dir()])
    selected_classes = np.random.choice(classes, min(num_samples, len(classes)), replace=False)
    
    fig, axes = plt.subplots(3, 4, figsize=figsize)
    axes = axes.ravel()
    
    for idx, class_name in enumerate(selected_classes):
        class_dir = directory / class_name
        images = list(class_dir.glob('*.jpg'))
        
        if images:
            random_image = np.random.choice(images)
            img = load_img(random_image, target_size=(IMG_HEIGHT, IMG_WIDTH))
            
            axes[idx].imshow(img)
            axes[idx].set_title(class_name, fontsize=10)
            axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Sample Butterfly Images', fontsize=16, y=1.02)
    plt.show()

plot_sample_images(TRAIN_DIR)

### 2.4 Class Distribution Analysis

In [ ]:
# Visualize class distribution
def plot_class_distribution(class_counts, title="Class Distribution", top_n=20):
    """Plot class distribution as a bar chart."""
    df = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Count'])
    df = df.sort_values('Count', ascending=False).head(top_n)
    
    plt.figure(figsize=(14, 6))
    plt.bar(range(len(df)), df['Count'], color='steelblue')
    plt.xlabel('Species', fontsize=12)
    plt.ylabel('Number of Images', fontsize=12)
    plt.title(f'{title} (Top {top_n})', fontsize=14)
    plt.xticks(range(len(df)), df['Class'], rotation=90, ha='right', fontsize=8)
    plt.tight_layout()
    plt.show()
    
    print(f"Mean images per class: {df['Count'].mean():.2f}")
    print(f"Std images per class: {df['Count'].std():.2f}")
    print(f"Min images per class: {df['Count'].min()}")
    print(f"Max images per class: {df['Count'].max()}")

plot_class_distribution(train_counts, "Training Set Class Distribution")

---
## 3. Data Preprocessing and Augmentation

### 3.1 Create Data Generators

We'll use `ImageDataGenerator` for:
1. **Normalization**: Scale pixel values to [0, 1]
2. **Data Augmentation**: Create variations of training images to improve generalization

In [ ]:
# Training data generator WITH augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values to [0, 1]
    rotation_range=20,           # Random rotation up to 20 degrees
    width_shift_range=0.2,       # Random horizontal shift
    height_shift_range=0.2,      # Random vertical shift
    shear_range=0.2,             # Shear transformation
    zoom_range=0.2,              # Random zoom
    horizontal_flip=True,        # Random horizontal flip
    fill_mode='nearest'          # Fill empty pixels after transformations
)

# Validation and test data generators WITHOUT augmentation (only rescaling)
valid_test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

validation_generator = valid_test_datagen.flow_from_directory(
    VALID_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = valid_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get class labels
class_labels = {v: k for k, v in train_generator.class_indices.items()}
num_classes = len(class_labels)

print(f"Number of classes: {num_classes}")
print(f"Training batches: {len(train_generator)}")
print(f"Validation batches: {len(validation_generator)}")
print(f"Test batches: {len(test_generator)}")

### 3.2 Visualize Data Augmentation

In [ ]:
def visualize_augmentation(generator, num_images=5):
    """Visualize augmented versions of a single image."""
    # Get a batch of images
    x_batch, y_batch = next(generator)
    
    # Take the first image
    img = x_batch[0]
    
    # Create augmented versions
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    
    axes[0].imshow(img)
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    # Generate augmented versions
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)
    
    aug_iter = train_datagen.flow(img_array, batch_size=1)
    
    for i in range(1, num_images):
        batch = next(aug_iter)
        aug_img = batch[0]
        axes[i].imshow(aug_img)
        axes[i].set_title(f'Augmented {i}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_augmentation(train_generator)

---
## 4. Building CNN Models

We'll build three different models with increasing complexity:
1. **Simple CNN**: Basic architecture for baseline
2. **Deeper CNN**: More layers and complexity
3. **Transfer Learning**: Using pre-trained models (VGG16, ResNet50, MobileNetV2)

### 4.1 Model 1: Simple CNN (Baseline)

In [ ]:
def create_simple_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), num_classes=num_classes):
    """
    Create a simple CNN model with 3 convolutional blocks.
    
    Architecture:
    - 3 Conv2D + MaxPooling blocks
    - Flatten
    - Dense layers with dropout
    """
    model = models.Sequential([
        # First convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third convolutional block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and fully connected layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create and display model
simple_cnn = create_simple_cnn()
simple_cnn.summary()

### 4.2 Model 2: Deeper CNN with Batch Normalization

In [ ]:
def create_deeper_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), num_classes=num_classes):
    """
    Create a deeper CNN model with batch normalization.
    
    Architecture:
    - 4 Conv2D blocks with BatchNormalization
    - MaxPooling after each block
    - Dropout for regularization
    - Dense layers with dropout
    """
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 4
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fully connected layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

deeper_cnn = create_deeper_cnn()
deeper_cnn.summary()

### 4.3 Model 3: Transfer Learning with Pre-trained Models

In [ ]:
def create_transfer_learning_model(base_model_name='MobileNetV2', 
                                   input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                                   num_classes=num_classes,
                                   trainable_layers=0):
    """
    Create a transfer learning model using pre-trained architectures.
    
    Args:
        base_model_name: 'VGG16', 'ResNet50', or 'MobileNetV2'
        input_shape: Input image shape
        num_classes: Number of output classes
        trainable_layers: Number of top layers to make trainable (0 = freeze all)
    """
    # Load pre-trained base model
    if base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Unknown base model: {base_model_name}")
    
    # Freeze base model layers
    base_model.trainable = False
    
    # If trainable_layers > 0, unfreeze top layers
    if trainable_layers > 0:
        base_model.trainable = True
        for layer in base_model.layers[:-trainable_layers]:
            layer.trainable = False
    
    # Build the model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Create transfer learning model with MobileNetV2
transfer_model = create_transfer_learning_model('MobileNetV2')
transfer_model.summary()

---
## 5. Model Compilation and Training

### 5.1 Define Training Callbacks

In [ ]:
def get_callbacks(model_name, patience=5):
    """
    Create callbacks for training.
    
    Args:
        model_name: Name for saving checkpoints
        patience: Number of epochs to wait before early stopping
    """
    callbacks = [
        # Early stopping: stop if validation loss doesn't improve
        EarlyStopping(
            monitor='val_loss',
            patience=patience,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Model checkpoint: save best model
        ModelCheckpoint(
            f'{model_name}_best.h5',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        
        # Reduce learning rate when plateauing
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    return callbacks

### 5.2 Train the Simple CNN Model

In [ ]:
# Compile the model
simple_cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
EPOCHS = 30

history_simple = simple_cnn.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=get_callbacks('simple_cnn', patience=5),
    verbose=1
)

### 5.3 Train the Deeper CNN Model

In [ ]:
# Compile the model
deeper_cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history_deeper = deeper_cnn.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=get_callbacks('deeper_cnn', patience=5),
    verbose=1
)

### 5.4 Train the Transfer Learning Model

In [ ]:
# Compile the model
transfer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate for transfer learning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history_transfer = transfer_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=get_callbacks('transfer_model', patience=7),
    verbose=1
)

---
## 6. Model Evaluation and Visualization

### 6.1 Plot Training History

In [ ]:
def plot_training_history(history, model_name):
    """
    Plot training and validation accuracy/loss over epochs.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Accuracy', fontsize=12)
    ax1.set_title(f'{model_name} - Accuracy', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss', marker='o')
    ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.set_title(f'{model_name} - Loss', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print(f"\n{model_name} - Final Metrics:")
    print(f"Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print(f"Training Loss: {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")

# Plot histories
plot_training_history(history_simple, 'Simple CNN')
plot_training_history(history_deeper, 'Deeper CNN')
plot_training_history(history_transfer, 'Transfer Learning (MobileNetV2)')

### 6.2 Compare All Models

In [ ]:
def compare_models(histories, model_names):
    """
    Compare validation accuracy of all models on the same plot.
    """
    plt.figure(figsize=(12, 6))
    
    for history, name in zip(histories, model_names):
        plt.plot(history.history['val_accuracy'], label=name, marker='o', linewidth=2)
    
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Validation Accuracy', fontsize=12)
    plt.title('Model Comparison - Validation Accuracy', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

compare_models(
    [history_simple, history_deeper, history_transfer],
    ['Simple CNN', 'Deeper CNN', 'Transfer Learning']
)

### 6.3 Evaluate on Test Set

In [ ]:
def evaluate_model(model, test_generator, model_name):
    """
    Evaluate model on test set and print metrics.
    """
    print(f"\n{'='*50}")
    print(f"Evaluating {model_name} on Test Set")
    print(f"{'='*50}")
    
    # Evaluate
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
    
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    
    return test_accuracy, test_loss

# Evaluate all models
simple_test_acc, simple_test_loss = evaluate_model(simple_cnn, test_generator, 'Simple CNN')
deeper_test_acc, deeper_test_loss = evaluate_model(deeper_cnn, test_generator, 'Deeper CNN')
transfer_test_acc, transfer_test_loss = evaluate_model(transfer_model, test_generator, 'Transfer Learning')

# Summary comparison
print("\n" + "="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)
print(f"{'Model':<30} {'Test Accuracy':<20} {'Test Loss':<20}")
print("-"*70)
print(f"{'Simple CNN':<30} {simple_test_acc:<20.4f} {simple_test_loss:<20.4f}")
print(f"{'Deeper CNN':<30} {deeper_test_acc:<20.4f} {deeper_test_loss:<20.4f}")
print(f"{'Transfer Learning':<30} {transfer_test_acc:<20.4f} {transfer_test_loss:<20.4f}")
print("="*70)

### 6.4 Generate Predictions and Classification Report

In [ ]:
# Use the best performing model (likely transfer learning)
best_model = transfer_model

# Get predictions
test_generator.reset()
predictions = best_model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_generator.classes

# Classification report
print("\nClassification Report (First 20 classes):")
print("="*80)
class_names = list(class_labels.values())
report = classification_report(true_classes, predicted_classes, target_names=class_names, zero_division=0)
print(report[:2000])  # Print first 2000 characters to avoid too much output

### 6.5 Confusion Matrix (Sample Classes)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, sample_size=20):
    """
    Plot confusion matrix for a sample of classes.
    """
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Sample classes for visualization (too many classes to show all)
    sample_indices = np.random.choice(len(class_names), min(sample_size, len(class_names)), replace=False)
    sample_indices = sorted(sample_indices)
    
    # Extract submatrix
    cm_sample = cm[np.ix_(sample_indices, sample_indices)]
    sample_class_names = [class_names[i] for i in sample_indices]
    
    # Plot
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm_sample, annot=True, fmt='d', cmap='Blues', 
                xticklabels=sample_class_names, yticklabels=sample_class_names,
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.title(f'Confusion Matrix (Sample of {sample_size} Classes)', fontsize=14)
    plt.xticks(rotation=90, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(true_classes, predicted_classes, class_names, sample_size=15)

### 6.6 Visualize Predictions

In [ ]:
def plot_predictions(model, generator, class_labels, num_samples=12, correct_only=False):
    """
    Visualize model predictions with confidence scores.
    
    Args:
        model: Trained model
        generator: Data generator
        class_labels: Dictionary mapping class indices to names
        num_samples: Number of samples to display
        correct_only: If True, show only correct predictions
    """
    # Get a batch
    generator.reset()
    x_batch, y_batch = next(generator)
    
    # Make predictions
    predictions = model.predict(x_batch, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_batch, axis=1)
    
    # Filter for correct/incorrect if needed
    if correct_only:
        indices = np.where(predicted_classes == true_classes)[0]
    else:
        indices = np.arange(len(predicted_classes))
    
    # Random sample
    sample_indices = np.random.choice(indices, min(num_samples, len(indices)), replace=False)
    
    # Plot
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, sample_idx in enumerate(sample_indices):
        img = x_batch[sample_idx]
        true_label = class_labels[true_classes[sample_idx]]
        pred_label = class_labels[predicted_classes[sample_idx]]
        confidence = predictions[sample_idx][predicted_classes[sample_idx]]
        
        # Display image
        axes[idx].imshow(img)
        
        # Color: green if correct, red if wrong
        color = 'green' if true_classes[sample_idx] == predicted_classes[sample_idx] else 'red'
        
        title = f"True: {true_label[:15]}\nPred: {pred_label[:15]}\nConf: {confidence:.2f}"
        axes[idx].set_title(title, fontsize=9, color=color)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Show predictions (mix of correct and incorrect)
plot_predictions(best_model, test_generator, class_labels, num_samples=12)

### 6.7 Analyze Misclassifications

In [ ]:
def analyze_misclassifications(true_classes, predicted_classes, class_labels, top_n=10):
    """
    Identify classes with highest misclassification rates.
    """
    # Calculate per-class accuracy
    class_accuracies = {}
    
    for class_idx in range(len(class_labels)):
        class_mask = true_classes == class_idx
        if np.sum(class_mask) > 0:
            correct = np.sum(predicted_classes[class_mask] == class_idx)
            total = np.sum(class_mask)
            accuracy = correct / total
            class_accuracies[class_labels[class_idx]] = accuracy
    
    # Sort by accuracy
    sorted_accuracies = sorted(class_accuracies.items(), key=lambda x: x[1])
    
    # Display worst performing classes
    print(f"\nTop {top_n} Worst Performing Classes:")
    print("="*60)
    print(f"{'Class Name':<40} {'Accuracy':<10}")
    print("-"*60)
    
    for class_name, acc in sorted_accuracies[:top_n]:
        print(f"{class_name:<40} {acc:<10.2%}")
    
    # Display best performing classes
    print(f"\nTop {top_n} Best Performing Classes:")
    print("="*60)
    print(f"{'Class Name':<40} {'Accuracy':<10}")
    print("-"*60)
    
    for class_name, acc in sorted_accuracies[-top_n:][::-1]:
        print(f"{class_name:<40} {acc:<10.2%}")

analyze_misclassifications(true_classes, predicted_classes, class_labels)

---
## 7. Fine-tuning Transfer Learning Model (Advanced)

After training with frozen base layers, we can fine-tune the model by unfreezing some layers.

In [ ]:
# Create a new transfer learning model with some unfrozen layers
fine_tune_model = create_transfer_learning_model('MobileNetV2', trainable_layers=20)

# Load weights from previous training (optional)
# fine_tune_model.load_weights('transfer_model_best.h5')

# Compile with lower learning rate
fine_tune_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),  # Very low learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Fine-tuning model with unfrozen layers...")
print(f"Total layers: {len(fine_tune_model.layers)}")
print(f"Trainable layers: {sum([layer.trainable for layer in fine_tune_model.layers])}")

# Train for fewer epochs
history_fine_tune = fine_tune_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=get_callbacks('fine_tune_model', patience=5),
    verbose=1
)

# Evaluate
fine_tune_test_acc, fine_tune_test_loss = evaluate_model(fine_tune_model, test_generator, 'Fine-tuned Model')

---
## 8. Save and Load Models

In [ ]:
# Save the best model
best_model.save('butterfly_classifier_final.h5')
print("Model saved successfully!")

# To load the model later:
# loaded_model = keras.models.load_model('butterfly_classifier_final.h5')

---
## 10. Prediction Function for New Images

In [ ]:
def predict_butterfly(image_path, model, class_labels, top_k=5):
    """
    Predict the species of a butterfly from an image file.
    
    Args:
        image_path: Path to the image file
        model: Trained model
        class_labels: Dictionary mapping class indices to names
        top_k: Number of top predictions to return
    """
    # Load and preprocess image
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)[0]
    
    # Get top k predictions
    top_indices = np.argsort(predictions)[-top_k:][::-1]
    top_predictions = [(class_labels[idx], predictions[idx]) for idx in top_indices]
    
    # Display results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Show image
    ax1.imshow(img)
    ax1.set_title('Input Image', fontsize=14)
    ax1.axis('off')
    
    # Show predictions
    species = [pred[0][:20] for pred in top_predictions]
    confidences = [pred[1] for pred in top_predictions]
    
    y_pos = np.arange(len(species))
    ax2.barh(y_pos, confidences, color='steelblue')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(species)
    ax2.invert_yaxis()
    ax2.set_xlabel('Confidence', fontsize=12)
    ax2.set_title(f'Top {top_k} Predictions', fontsize=14)
    ax2.set_xlim([0, 1])
    
    # Add confidence values
    for i, v in enumerate(confidences):
        ax2.text(v + 0.01, i, f'{v:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    return top_predictions

# Example usage (uncomment and provide a valid image path):
# predictions = predict_butterfly('path/to/butterfly/image.jpg', best_model, class_labels)